https://blog.naver.com/PostView.naver?blogId=zzing0907&logNo=221612308385

In [5]:
import cv2
import dlib
import time

# Load the detectors and predictors
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./models/shape_predictor_68_face_landmarks.dat")

def detect_face_68(frame_bgr):
    gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    for (i, rect) in enumerate(rects):
        x1 = rect.left()
        y1 = rect.top()
        x2 = rect.right()
        y2 = rect.bottom()
        cv2.rectangle(frame_bgr, (x1, y1), (x2, y2), (0,255,0), 2)

        # Get the landmarks/parts for the face in box rect.
        shape = predictor(gray, rect)

        # Draw the face landmarks on the screen.
        for i in range(0, shape.num_parts):
            part = shape.part(i)
            cv2.circle(frame_bgr, (part.x, part.y), 2, (0, 0, 255), -1)

    return frame_bgr

# video

In [6]:
video_capture = cv2.VideoCapture(0)
s_time = time.time()
cnt = 0

while True:
    cnt += 1
    ret, frame_bgr = video_capture.read()
    frame_bgr = detect_face_68(frame_bgr)
    
    cv2.imshow('Video', frame_bgr)
    if cv2.waitKey(1)==27:  # 27 == Esc
        break
    
    if time.time() - s_time > 0.1:
        print(f"\rFPS = {cnt / (time.time() - s_time):.1f}" + ' ' * 20, end = '')
        s_time = time.time()
        cnt = 0

video_capture.release()
cv2.destroyAllWindows()

FPS = 8.9                    

You only Look Once (YOLO)
- opencv YOLO 3 320x320, 609x609, 416x416
https://opencv-tutorial.readthedocs.io/en/latest/yolo/yolo.html 
- YOLO 8

In [ ]:
yolo path 



14 X 14, 28 x 28, 56 x 56
each cell in the frature map has 3 boxes and each scale has the boxes of 588, 2352, 9408
- 588= 14 x 14 x 3
- 2352 = 28 x 28 x 3
- 9408 = 56 x 56 x 3

80개의 객체, 박스 의 중심x,중심y,w,h, 객체일 확률

Non-Maximum suppression (NMS) is used to find one box from many overlapping boxes

B= 박스들 ,S = 임계치 이하의 박스들을 제거후에 남은 박스들
F - () 살릴놈들
m - 제일점수가 높은놈 한놈 고름
그 박스의 점수와 임계치 값을 고른후 둘중에 큰 값을 고름
나온 값을 살릴놈들에 집어 넣음
B- m 을 뺀다
남아 있는 S들 중에서 IoU를 계산하여 많이 높으면 지운다. 지운거는 F에 안 넣는다.(0~1) = Area of intersection / Area of Union(점수가 높을 수록 많이 겹친다.) IoU를 계산하여 낮은것을 살린다.

cv2.dnn.NMSBoxes(bboxes, scores, score_threshold, nms_threshold)


In [ ]:
# cv2.dnn.NMSBoxes(bboxes, scores, score_threshold, nms_threshold)

box = 

# Yolo8
- https://developer-lionhong.tistory.com/62

In [3]:
print(torch.cuda.is_available())

False


In [ ]:
from ultralytics import YOLO
# import torch

yolo8_model = YOLO('./models/yolov8m.pt')
yolo8_model.to('cuda')

In [ ]:
import cv2
import time
import numpy as np
from ultralytics import YOLO
import torch

# Initialize video capture
video_capture = cv2.VideoCapture("humans.mp4")

# YOLO model and parameters
conf_score_th = 0.5
iou_th = 0.4
# Assume yolo8_model is initialized elsewhere in your code

# Initialize counters and timing
frame_cnt = 0
s_time = time.time()
fps = 0
cnt = 0

while True:
    ret, frame_bgr = video_capture.read()
    if not ret:
        break
    
    # Perform object detection
    detection = yolo8_model.predict(frame_bgr, iou=iou_th, conf=conf_score_th, verbose=False)[0]
    frame_bgr = detection.plot()
    
    # Display the frame
    cv2.imshow('Video', frame_bgr)
    
    # FPS calculation
    cnt += 1
    if time.time() - s_time > 0.1:
        print(f"\rFPS = {cnt / (time.time() - s_time):.1f}" + ' ' * 20, end = '')
        s_time = time.time()
        cnt = 0
    
    # Break the loop if 'Esc' key is pressed
    if cv2.waitKey(1) == 27:  # 27 is the ASCII code for the 'Esc' key
        break

# Release the video capture object and close display windows
video_capture.release()
cv2.destroyAllWindows()
